In [1]:
import numpy as np
import pandas as pd

from ols_bootstrap.pairs import PairsBootstrap
from ols_bootstrap.residual import ResidualBootstrap
from ols_bootstrap.wild import WildBootstrap

pd.options.display.float_format = '{:20,.5f}'.format  ### Setting pd to have a numerical precision up to 5 decimal points

In [2]:
df = pd.read_csv('./balance2018.csv')
df = df[df['sales_clean'] != 0]
df = df[['sales_clean', 'tanass_clean', 'tax']]
df = df.dropna(subset=['tanass_clean', 'tax'])

df_scaled = df.applymap(lambda x: np.log(x + 1))

/home/phamv/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (66) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_sample = df_scaled.sample(n=10000, replace=False, random_state=42)

Y_data = pd.DataFrame(df_sample.iloc[:, 0])
X_data = pd.DataFrame(df_sample.iloc[:, 1:])

## Default SE on the original OLS is HC3, default CI on bootstrapped parameter is BC. 

That is by default se_type = 'hc3', ci_type = 'bc'.

The default seed value is None. For reproducability, use an integer of your choice. 

In [4]:
psb = PairsBootstrap(Y_data, X_data, reps = 1000, se_type='hc3', ci_type = 'bc', seed = 42)  # seed = None, se_type = 'hc3' and ci_type = 'bc' are the default options for these arguments.
psb.fit()

In [5]:
rsb_not_scaled = ResidualBootstrap(Y_data, X_data, reps = 1000, scale_resid_bool  = False, seed = 42)
rsb_not_scaled.fit()

In [6]:
rsb = ResidualBootstrap(Y_data, X_data, reps = 1000, seed = 42)
rsb.fit()

In [7]:
wb_unif = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "uniform", seed = 42)
wb_unif.fit()

In [8]:
wb_stdn = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "standard_normal", seed = 42)
wb_stdn.fit()

In [9]:
wb_rad_not_scaled = WildBootstrap(Y_data, X_data, reps = 1000, scale_resid_bool = False, from_distro = "rademacher", seed = 42)
wb_rad_not_scaled.fit()

In [10]:
wb_rad = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "rademacher", seed = 42)
wb_rad.fit()

In [11]:
wb_rad_bca = WildBootstrap(Y_data, X_data, reps = 1000, ci_type='bca', from_distro = "rademacher", seed = 42)
wb_rad_bca.fit()

In [12]:
wb_rad_emp = WildBootstrap(Y_data, X_data, reps = 1000, ci_type='empirical', from_distro = "rademacher", seed = 42)
wb_rad_emp.fit()

In [13]:
wb_rad_perc = WildBootstrap(Y_data, X_data, reps = 1000, ci_type='percentile', from_distro = "rademacher", seed = 42)
wb_rad_perc.fit()

In [14]:
wb_rad_basic = WildBootstrap(Y_data, X_data, reps = 1000, ci_type='basic', from_distro = "rademacher", seed = 42)
wb_rad_basic.fit()

In [15]:
wb_webb4 = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "webb4", seed = 42)
wb_webb4.fit()

In [16]:
wb_webb6 = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "webb6", seed = 42)
wb_webb6.fit()

In [17]:
wb_cont_mam = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "mammen_cont", seed = 42)
wb_cont_mam.fit()

In [18]:
wb_mam = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "mammen", seed = 42)

Before running the bootstrapping, we can yield the simple OLS result (using statsmodel API for the table summary formula), conduct White test, get all versions of standard errors.  

In [19]:
wb_mam.ols_fit_sample()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.552
Model:                            OLS   Adj. R-squared:                  0.552
Method:                 Least Squares   F-statistic:                     4282.
Date:                Wed, 05 Oct 2022   Prob (F-statistic):               0.00
Time:                        16:53:54   Log-Likelihood:                -18267.
No. Observations:               10000   AIC:                         3.654e+04
Df Residuals:                    9997   BIC:                         3.656e+04
Df Model:                           2                                         
Covariance Type:                  HC3                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            6.2070      0.045    136.781   

In [20]:
wb_mam.white_test()

(1439.4667489471435, 3.861829429235033e-309, 336.10127470057813, 0.0)

get_all_se() method

The following SE-s are calculated: 
- bootstrapped - standard error of the bootstrapped parameters
- constant - model-based OLS Standard Errors, that is, constant variance is assumed 
- HC0, HC1, HC2, HC3, HC4, HC4m, HC5 - Heteroskedasticity-Consistent Standard Errors (HCE) using sandwich estimators 

In [21]:
wb_mam.get_all_se()

,nonrobust,hc0,hc1,hc2,hc3,hc4,hc4m,hc5
const,0.03532,0.04536,0.04536,0.04537,0.04538,0.04538,0.04538,0.04538
tanass_clean,0.00391,0.00499,0.00499,0.00499,0.00499,0.00499,0.00499,0.00499
tax,0.00657,0.00910,0.00910,0.00911,0.00911,0.00911,0.00911,0.00911


In [22]:
wb_mam.get_all_se(which_var=['tanass_clean', 'tax'])

,nonrobust,hc0,hc1,hc2,hc3,hc4,hc4m,hc5
tanass_clean,0.00391,0.00499,0.00499,0.00499,0.00499,0.00499,0.00499,0.00499
tax,0.00657,0.00910,0.00910,0.00911,0.00911,0.00911,0.00911,0.00911


In [23]:
wb_mam.get_all_se(which_var='tax')

,nonrobust,hc0,hc1,hc2,hc3,hc4,hc4m,hc5
tax,0.00657,0.00910,0.00910,0.00911,0.00911,0.00911,0.00911,0.00911


In [24]:
wb_mam.fit()

In [25]:
# Now: np.ndarray was supported as an input array. pd.Series, list, and tuples were also added to the supported list for an input array. 
# Before that only pd.DataFrame had been supported. 
X_data_np = X_data.to_numpy()
Y_data_np = Y_data.to_numpy()

In [26]:
wb_mam_np = WildBootstrap(Y_data_np, X_data_np, reps = 1000, from_distro = "mammen", seed = 42)
wb_mam_np.fit()

In [27]:
psb.summary()

,ols_params,avg_bs_params,bias,ols_params_se,bs_params_se,perc_of_se_diff,ci_lwb,ci_upb
var,,,,,,,,
const,6.207,6.207,0.0,0.0454,0.0462,-1.7,6.1273,6.3045
tanass_clean,0.1693,0.1691,0.0002,0.005,0.0051,-1.56,0.1591,0.1785
tax,0.5293,0.5296,0.0003,0.0091,0.009,1.12,0.5106,0.5455


In [29]:
rsb_not_scaled.summary()

,ols_params,avg_bs_params,bias,ols_params_se,bs_params_se,perc_of_se_diff,ci_lwb,ci_upb
var,,,,,,,,
const,6.207,6.2058,0.0012,0.0454,0.0349,23.0,6.142,6.2788
tanass_clean,0.1693,0.1694,0.0001,0.005,0.0039,22.09,0.1617,0.1772
tax,0.5293,0.5293,0.0,0.0091,0.0064,29.36,0.5166,0.5425


In [30]:
rsb.summary()

,ols_params,avg_bs_params,bias,ols_params_se,bs_params_se,perc_of_se_diff,ci_lwb,ci_upb
var,,,,,,,,
const,6.207,6.2058,0.0011,0.0454,0.035,22.97,6.1421,6.2789
tanass_clean,0.1693,0.1694,0.0001,0.005,0.0039,22.06,0.1617,0.1772
tax,0.5293,0.5293,0.0,0.0091,0.0064,29.33,0.5166,0.5425


In [31]:
wb_unif.summary()

,ols_params,avg_bs_params,bias,ols_params_se,bs_params_se,perc_of_se_diff,ci_lwb,ci_upb
var,,,,,,,,
const,6.207,6.2082,0.0013,0.0454,0.0447,1.43,6.1199,6.2936
tanass_clean,0.1693,0.1693,0.0,0.005,0.0051,-3.14,0.1597,0.1794
tax,0.5293,0.529,0.0003,0.0091,0.0092,-0.79,0.5121,0.5486


In [32]:
wb_stdn.summary()

,ols_params,avg_bs_params,bias,ols_params_se,bs_params_se,perc_of_se_diff,ci_lwb,ci_upb
var,,,,,,,,
const,6.207,6.2059,0.0011,0.0454,0.0479,-5.48,6.1209,6.3076
tanass_clean,0.1693,0.1689,0.0004,0.005,0.005,-0.84,0.1595,0.1785
tax,0.5293,0.5301,0.0008,0.0091,0.0093,-2.58,0.5095,0.5468


In [33]:
wb_rad_not_scaled.summary()

,ols_params,avg_bs_params,bias,ols_params_se,bs_params_se,perc_of_se_diff,ci_lwb,ci_upb
var,,,,,,,,
const,6.207,6.2042,0.0028,0.0454,0.0446,1.61,6.1227,6.3006
tanass_clean,0.1693,0.1694,0.0001,0.005,0.0051,-2.73,0.1592,0.1791
tax,0.5293,0.5296,0.0003,0.0091,0.0092,-0.78,0.5105,0.5465


In [34]:
wb_rad.summary()

,ols_params,avg_bs_params,bias,ols_params_se,bs_params_se,perc_of_se_diff,ci_lwb,ci_upb
var,,,,,,,,
const,6.207,6.2042,0.0028,0.0454,0.0447,1.56,6.1227,6.3007
tanass_clean,0.1693,0.1694,0.0001,0.005,0.0051,-2.8,0.1592,0.1791
tax,0.5293,0.5296,0.0003,0.0091,0.0092,-0.85,0.5105,0.5465


In [35]:
wb_rad_bca.summary()

,ols_params,avg_bs_params,bias,ols_params_se,bs_params_se,perc_of_se_diff,ci_lwb,ci_upb
var,,,,,,,,
const,6.207,6.2042,0.0028,0.0454,0.0447,1.56,6.1228,6.3009
tanass_clean,0.1693,0.1694,0.0001,0.005,0.0051,-2.8,0.1592,0.1791
tax,0.5293,0.5296,0.0003,0.0091,0.0092,-0.85,0.5105,0.5463


In [36]:
wb_rad_bca_jack = WildBootstrap(Y_data, X_data, reps = 1000, ci_type='bca', from_distro = "rademacher", subset_jack_ratio=0.5, seed = 42) #using partial jacknife estimation for a_hat acceleration param 
wb_rad_bca_jack.fit()
wb_rad_bca_jack.summary()

,ols_params,avg_bs_params,bias,ols_params_se,bs_params_se,perc_of_se_diff,ci_lwb,ci_upb
var,,,,,,,,
const,6.207,6.2042,0.0028,0.0454,0.0447,1.56,6.123,6.3012
tanass_clean,0.1693,0.1694,0.0001,0.005,0.0051,-2.8,0.1592,0.1791
tax,0.5293,0.5296,0.0003,0.0091,0.0092,-0.85,0.5104,0.5463


In [37]:
wb_rad_emp.summary()

,ols_params,avg_bs_params,bias,ols_params_se,bs_params_se,perc_of_se_diff,ci_lwb,ci_upb
var,,,,,,,,
const,6.207,6.2042,0.0028,0.0454,0.0447,1.56,6.1234,6.2984
tanass_clean,0.1693,0.1694,0.0001,0.005,0.0051,-2.8,0.1594,0.1793
tax,0.5293,0.5296,0.0003,0.0091,0.0092,-0.85,0.5109,0.5473


In [38]:
wb_rad_perc.summary()

,ols_params,avg_bs_params,bias,ols_params_se,bs_params_se,perc_of_se_diff,ci_lwb,ci_upb
var,,,,,,,,
const,6.207,6.2042,0.0028,0.0454,0.0447,1.56,6.1155,6.2905
tanass_clean,0.1693,0.1694,0.0001,0.005,0.0051,-2.8,0.1593,0.1791
tax,0.5293,0.5296,0.0003,0.0091,0.0092,-0.85,0.5113,0.5477


In [39]:
wb_rad_basic.summary()

,ols_params,avg_bs_params,bias,ols_params_se,bs_params_se,perc_of_se_diff,ci_lwb,ci_upb
var,,,,,,,,
const,6.207,6.2042,0.0028,0.0454,0.0447,1.56,6.1234,6.2984
tanass_clean,0.1693,0.1694,0.0001,0.005,0.0051,-2.8,0.1594,0.1793
tax,0.5293,0.5296,0.0003,0.0091,0.0092,-0.85,0.5109,0.5473


In [40]:
wb_webb4.summary()

,ols_params,avg_bs_params,bias,ols_params_se,bs_params_se,perc_of_se_diff,ci_lwb,ci_upb
var,,,,,,,,
const,6.207,6.2091,0.0021,0.0454,0.0446,1.69,6.119,6.2955
tanass_clean,0.1693,0.1692,0.0001,0.005,0.0051,-3.15,0.1599,0.1799
tax,0.5293,0.529,0.0003,0.0091,0.0091,-0.3,0.5119,0.5479


In [41]:
wb_webb6.summary()

,ols_params,avg_bs_params,bias,ols_params_se,bs_params_se,perc_of_se_diff,ci_lwb,ci_upb
var,,,,,,,,
const,6.207,6.2093,0.0023,0.0454,0.0445,1.95,6.1193,6.2944
tanass_clean,0.1693,0.1692,0.0001,0.005,0.0051,-2.77,0.1595,0.1794
tax,0.5293,0.529,0.0003,0.0091,0.0091,-0.37,0.5123,0.5479


In [42]:
wb_cont_mam.summary()

,ols_params,avg_bs_params,bias,ols_params_se,bs_params_se,perc_of_se_diff,ci_lwb,ci_upb
var,,,,,,,,
const,6.207,6.206,0.0009,0.0454,0.0465,-2.53,6.1229,6.3002
tanass_clean,0.1693,0.1691,0.0002,0.005,0.0052,-4.57,0.1595,0.1791
tax,0.5293,0.5297,0.0004,0.0091,0.0093,-2.35,0.5116,0.5478


In [43]:
wb_mam.summary()

,ols_params,avg_bs_params,bias,ols_params_se,bs_params_se,perc_of_se_diff,ci_lwb,ci_upb
var,,,,,,,,
const,6.207,6.2068,0.0002,0.0454,0.0462,-1.8,6.1173,6.3004
tanass_clean,0.1693,0.1692,0.0,0.005,0.0052,-3.34,0.1591,0.1788
tax,0.5293,0.5294,0.0001,0.0091,0.0093,-2.59,0.5117,0.547


Same output for wb_mam_np as in the case for wb_mam, but the difference is the independent variables' name. wb_mam was constructed with pd.DataFrame the input which holds information about the column name. 
When it comes to wb_mam_np, it was constructed with np.ndarray as the input which DOES NOT hold info about the column name originally. Because of this, x1, x2, ...., xn were generated as alternative independent variable names. 

In [44]:
wb_mam_np.summary()

,ols_params,avg_bs_params,bias,ols_params_se,bs_params_se,perc_of_se_diff,ci_lwb,ci_upb
var,,,,,,,,
const,6.207,6.2068,0.0002,0.0454,0.0462,-1.8,6.1173,6.3004
x1,0.1693,0.1692,0.0,0.005,0.0052,-3.34,0.1591,0.1788
x2,0.5293,0.5294,0.0001,0.0091,0.0093,-2.59,0.5117,0.547


## Some useful methods were implemented (let's use it on wb_mam object of Wild Bootstrap with Mammen)

- The common in the following three methods is that either a string (if one wishes to capture one variable or ci) or 1-D like array can be provided to 'which_var' and/or 'which_ci' (the latter if exists in that object class).

### get_ci() method

#### Vanila version is when only the actual CI was used with all independent variables

In [45]:
wb_mam.get_bca_ci()

bc                     
                              lwb                  upb
const                     6.11727              6.30036
tanass_clean              0.15909              0.17883
tax                       0.51171              0.54702

#### However, any combination of ('bc', 'bca', 'percentile) CI types could be selected and any combinations of independent variables can be chosen with 'which_ci' and 'which_var' optional arguments, respectively.

Please note that if choosing 'bca' the calculation can take a while as it uses jacknife resampling for calculating the acceleration factor

In [46]:
wb_mam.get_bca_ci(which_ci='all', which_var='all')

basic                                        bc  \
                              lwb                  upb                  lwb   
const                     6.11362              6.29675              6.11727   
tanass_clean              0.15954              0.17916              0.15909   
tax                       0.51160              0.54693              0.51171   

                                                   bca                       \
                              upb                  lwb                  upb   
const                     6.30036              6.11732              6.30039   
tanass_clean              0.17883              0.15920              0.17886   
tax                       0.54702              0.51144              0.54689   

                        empirical                                percentile  \
                              lwb                  upb                  lwb   
const                     6.11362              6.29675              6.11716   
tanass_clean              0.15954              0.17916              0.15943   
tax                       0.51160              0.54693              0.51166   

                                   
                              upb  
const                     6.30029  
tanass_clean              0.17904  
tax                       0.54699

In [47]:
wb_mam.get_bca_ci(which_ci=['bc', 'percentile'], which_var=['tax', 'const'])

bc                                percentile  \
                       lwb                  upb                  lwb   
tax                0.51171              0.54702              0.51166   
const              6.11727              6.30036              6.11716   

                            
                       upb  
tax                0.54699  
const              6.30029

In [48]:
wb_mam.get_bca_ci(which_ci=['bca', 'bc'], which_var='tanass_clean')

bc                                       bca  \
                              lwb                  upb                  lwb   
tanass_clean              0.15909              0.17883              0.15920   

                                   
                              upb  
tanass_clean              0.17886

In [49]:
wb_mam.get_bca_ci(which_ci='bc', which_var='tanass_clean')

bc                     
                              lwb                  upb
tanass_clean              0.15909              0.17883

### get_bootstrap_params() method

#### Vanila version: Returning a dataframe capturing the parameter estimate of ALL each independent variables in each (wild) bootstrap.

In [50]:
wb_mam.get_bootstrap_params()

,const,tanass_clean,tax
0,6.12842,0.17666,0.53513
1,6.14828,0.17589,0.52739
2,6.16227,0.17062,0.53366
3,6.26953,0.16372,0.52545
4,6.22074,0.16961,0.52394
...,...,...,...
995,6.17475,0.17185,0.52915
996,6.19432,0.17210,0.52283
997,6.20963,0.17165,0.52893
998,6.19526,0.16878,0.53376


#### As usual, the desired independent variable can be chosen with 'which_var' argument

In [51]:
wb_mam.get_bootstrap_params(which_var='tax')

,tax
0,0.53513
1,0.52739
2,0.53366
3,0.52545
4,0.52394
...,...
995,0.52915
996,0.52283
997,0.52893
998,0.53376


In [52]:
wb_mam.get_bootstrap_params(which_var=('const', 'tanass_clean'))

,const,tanass_clean
0,6.12842,0.17666
1,6.14828,0.17589
2,6.16227,0.17062
3,6.26953,0.16372
4,6.22074,0.16961
...,...,...
995,6.17475,0.17185
996,6.19432,0.17210
997,6.20963,0.17165
998,6.19526,0.16878


### bp_test() and white_test() methods

#### Vanila version: for bp_test() returning the robust version (proposed by Roger Koenker) of this test when the presence of the heteroscedasticity is examined on the residual of the OLS on the original sample data. For white_test(), there are no optional arguments

In [53]:
wb_mam.bp_test()

(432.63698691408894,
 1.1325880054877581e-94,
 226.03260440021265,
 9.77405692800898e-97)

In [54]:
wb_mam.white_test()

(1439.4667489471435, 3.861829429235033e-309, 336.10127470057813, 0.0)

#### The non-robust version of bp_test() can be achieved by setting bp_test(robust = False).

In [55]:
wb_mam.bp_test(robust = False)

(1037.2017788129997,
 5.9497048313734906e-226,
 226.03260440020952,
 9.774056928031758e-97)

At each test, the first value is the calculated LM-test stats, the second is the 'LM p-value' corresponding to the LM-test stats, the third value is the F-test stats and last one is the 'F p-value'.

As it can be seen the p-values are 0, so we can rejcet the null hypothesis of homoscedasticity, and accept the alternative hypothesis that the variance of the residuals is heteroscedastic.

### Some built-in attributes

In [56]:
wb_mam.indep_varname

['const', 'tanass_clean', 'tax']

#### wb_mam.bs_params and wb_mam.get_bootstrap_params() basically output the same thing with the exception that the latter one is transposed and is in pd.DataFrame.

In [57]:
wb_mam.bs_params

array([[6.12841699, 6.1482837 , 6.16227211, ..., 6.20962585, 6.19526111,
        6.25780712],
       [0.17665865, 0.17589137, 0.17061916, ..., 0.17164619, 0.16878034,
        0.17450279],
       [0.53512822, 0.52739329, 0.53366386, ..., 0.52892758, 0.53375909,
        0.51448682]])

In [58]:
wb_mam.orig_params

array([6.20695725, 0.16929278, 0.52929771])

In [59]:
wb_mam.bs_mean

array([6.20678843, 0.16924468, 0.52938633])

In [60]:
wb_mam.orig_se

array([0.0453789 , 0.00499137, 0.00910921])

In [61]:
wb_mam.bs_se

array([0.04619795, 0.00515802, 0.00934533])

In [62]:
wb_mam.bs_ci

array([[6.11726794, 6.30036258],
       [0.15909215, 0.17883297],
       [0.51171085, 0.54701509]])